In [1]:
from utils.finalyca_store import get_finalyca_scoped_session
from utils.utils import print_query
from fin_models.transaction_models import NAV, RiskFreeIndexRate
from bizlogic.analytics import get_risk_ratios
import datetime
from sqlalchemy import func
import pandas as pd
import numpy as np
from bizlogic.core_helper import *
import time

In [2]:
from compass.investor_portfolio import *

In [3]:
db_session = get_finalyca_scoped_session(False)
TRANSACTION_DATE = datetime.datetime(2022, 5, 31)
account_ids = [757, 751]
portfolio_date= datetime.date(2022, 10, 31)

In [4]:
holdings_df = get_normalized_portfolio_holdings(db_session, account_ids, portfolio_date, False)

In [5]:
start = time.time()
performance_date = get_portfolio_performance_analysis_by_date(db_session, holdings_df, datetime.date(2022, 11, 30), datetime.date(2022, 12, 31))
print(F"Time Taken: {time.time() - start} Seconds")

Time Taken: 1.2315983772277832 Seconds


In [6]:
start = time.time()
performance_return = get_portfolio_performance_analysis_for_all_period(db_session, holdings_df, datetime.date(2022, 12, 31), ["1m"])
print(F"Time Taken: {time.time() - start} Seconds")

Time Taken: 0.46573901176452637 Seconds


In [7]:
isin_series = holdings_df["isin"]
units_series = holdings_df["units"]
start_date = datetime.date(2021, 10, 31)
end_date = datetime.date(2022, 10, 31)
end_price_df = None

df1 = pd.DataFrame(columns = ['isin', 'units'])
df1["isin"] = isin_series
df1["units"] = units_series

isin_list = list(df1["isin"].unique())

df_price_start_dt = get_portfolio_price_df(db_session, isin_list, start_date)
df_price_start_dt.rename(columns = {'date':'start_date', 'unit_price':'start_unit_price'}, inplace=True)

if end_date:
    df_price_end_dt = get_portfolio_price_df(db_session, isin_list, end_date)
    df_price_end_dt.rename(columns = {'date':'end_date', 'unit_price':'end_unit_price'}, inplace=True)
else:
    df_price_end_dt = end_prices_df

df = df1.merge(df_price_start_dt, on=["isin"], how='left')
df = df.merge(df_price_end_dt, on=['lookup_code', "isin"], how='left')

In [8]:
# Find list of securities that have end price but not start price. (security was listed after the start date)
missing = df.loc[df["start_unit_price"].isnull() & ~df["end_unit_price"].isnull()]

 # TODO: Check differently for closing and opening prices
lst_eq_co_codes = missing[missing['isin'].str.startswith('INE')]['lookup_code'].to_list()
lst_mf_plan_ids = missing[missing['isin'].str.startswith('INF')]['lookup_code'].to_list()
df_missing_prices = get_missing_security_prices(db_session, lst_eq_co_codes, start_date) if len(lst_eq_co_codes) > 0 else pd.DataFrame()
df_missing_mf_pr = get_missing_mf_prices(db_session, lst_mf_plan_ids, start_date) if len(lst_mf_plan_ids) > 0 else pd.DataFrame()
df_missing_prices = pd.concat([df_missing_prices, df_missing_mf_pr])
df_missing_prices.rename(columns={
    'date': 'start_date',
    'price': 'start_unit_price',
}, inplace=True)
df_missing_prices['lookup_code'] = df_missing_prices['lookup_code'].astype(str)

In [9]:
df

,isin,units,lookup_code,start_date,start_unit_price,end_date,end_unit_price
0,INF789F1AUX7,5400.000,42897,2021-10-31,None,2022-10-31,42.964700000000
1,INF174KA1HJ8,4500.000,36179,2021-10-31,None,2022-10-31,43.139700000000
2,INF200KA16D8,4000.000,119,2021-10-31,None,2022-10-31,44.267900000000
3,INF179KC1981,3600.000,1495,2021-10-31,None,2022-10-31,44.194600000000
4,INF846K01W80,3600.000,2667,2021-10-31,None,2022-10-31,43.114900000000
5,INF966L01580,727.510,8923,2021-10-31,None,2022-10-31,88.990100000000
6,INF179K01WA6,189.564,8359,2021-10-31,None,2022-10-31,338.291000000000
7,INF109K012B0,1080.067,5690,2021-10-31,None,2022-10-31,57.700000000000
8,INF204K01G94,447.477,9382,2021-10-31,None,2022-10-31,139.418300000000
9,INF754K01BS2,1550.331,10034,2021-10-31,None,2022-10-31,40.330000000000


In [10]:
df_missing_prices

,lookup_code,start_date,start_unit_price
0,42897,2021-10-31,41.5504
1,8359,2021-10-31,296.1300
2,36179,2021-10-31,41.5316
3,1495,2021-10-31,42.5643
4,8923,2021-10-31,76.3333
5,10034,2021-10-31,39.0300
6,119,2021-10-31,4262.6051
7,2667,2021-10-31,41.4301
8,9382,2021-10-31,131.7333
9,5690,2021-10-31,53.5300


In [11]:
# if not df_missing_prices.empty:
#     # df_missing_prices['lookup_code_x'] = df_missing_prices['lookup_code_x'].astype(str)
#     # df_missing_prices['lookup_code_y'] = df_missing_prices['lookup_code_x']
    # df_missing_prices.set_index('lookup_code', inplace=True)
    # df.set_index('lookup_code', inplace=True)
    # df["start_unit_price"].fillna(df_missing_prices["start_unit_price"], inplace=True)    
#     df.update(df_missing_prices)
    # df.reset_index(inplace=True)

# df = df.apply(pd.to_numeric, errors='coerce').fillna(df)

# df["start_price"] = df["units"]*df["start_unit_price"]
# df["end_price"] = df["units"]*df["end_unit_price"]
# df["change_pr"] = ((df["end_price"] - df["start_price"])/df["start_price"])*100
# df = df.fillna(np.nan).replace([np.nan], [None])

In [12]:
df_missing_prices.set_index('lookup_code', inplace=True)
df.set_index('lookup_code', inplace=True)

In [13]:
df_missing_prices.reset_index()

,lookup_code,start_date,start_unit_price
0,42897,2021-10-31,41.5504
1,8359,2021-10-31,296.1300
2,36179,2021-10-31,41.5316
3,1495,2021-10-31,42.5643
4,8923,2021-10-31,76.3333
5,10034,2021-10-31,39.0300
6,119,2021-10-31,4262.6051
7,2667,2021-10-31,41.4301
8,9382,2021-10-31,131.7333
9,5690,2021-10-31,53.5300


In [14]:
df.update(df_missing_prices)
# df["start_unit_price"].fillna(df_missing_prices["start_unit_price"], inplace=True)    

In [15]:
df

,isin,units,start_date,start_unit_price,end_date,end_unit_price
lookup_code,,,,,,
42897,INF789F1AUX7,5400.000,2021-10-31,41.5504,2022-10-31,42.964700000000
36179,INF174KA1HJ8,4500.000,2021-10-31,41.5316,2022-10-31,43.139700000000
119,INF200KA16D8,4000.000,2021-10-31,4262.6051,2022-10-31,44.267900000000
1495,INF179KC1981,3600.000,2021-10-31,42.5643,2022-10-31,44.194600000000
2667,INF846K01W80,3600.000,2021-10-31,41.4301,2022-10-31,43.114900000000
8923,INF966L01580,727.510,2021-10-31,76.3333,2022-10-31,88.990100000000
8359,INF179K01WA6,189.564,2021-10-31,296.13,2022-10-31,338.291000000000
5690,INF109K012B0,1080.067,2021-10-31,53.53,2022-10-31,57.700000000000
9382,INF204K01G94,447.477,2021-10-31,131.7333,2022-10-31,139.418300000000


In [16]:
df_missing_prices

,start_date,start_unit_price
lookup_code,,
42897,2021-10-31,41.5504
8359,2021-10-31,296.1300
36179,2021-10-31,41.5316
1495,2021-10-31,42.5643
8923,2021-10-31,76.3333
10034,2021-10-31,39.0300
119,2021-10-31,4262.6051
2667,2021-10-31,41.4301
9382,2021-10-31,131.7333
